<div class="text-muted">

# Lecture Notes

Instructions:

> **Week k (week that group topics are presented in lecture)** <br>
By Friday at 4pm, the group is required to submit their typed-up notes from that week’s
lecture. This will form the basic outline of their deliverable and they will receive
feedback from the TA on their work. <br><br>
The group should also have an outline of the one outside paper that they intend to
describe in greater detail.

## L13. Constraint Programming
- Big picture
- Viewing Sodoku and n-Queens as CSPs
- Constraint Modeling
- Applications of CSPs
- Tying to other course content

## L14. Solving Constraint Programs - Propagation and Basic Search
- Constraint propagation: Achieving arc consistency with AC-1 and AC-3
- Examples on integer domains with algebra
- Complexity of AC-1 and AC-3
- Arc consistency: sound but not complete

## L15. Solving Constraint Programs - Backtrack Search and Forward Checking
- Narrative: show the largest reasonable n-Queens problem that can be solved for various solution methods
- Combining inference with search
    - Why standard search sucks
    - Backtrack search
- Constraint Optimization Problem: Branch & Bound
- Forward Checking: Combining backtracking with **limited** constraint propagation
- Dynamic variable ordering and BT-FC
- "Bucket Elimination", a form of adaptive consistency (i.e., variable elimination)
    - Viewing arc consistency as variable elimination

###### Optional Content, Not Discussed in Lecture
- Adaptive consistency is linear for trees
- Iterative repair
- Conflict-directed back jumping

</div>

---

# Constraint Programming
Topics: constraint propagation, search, variable elimination; global constraints; MVRP

## 1. Introduction

Constraints naturally arise in a variety of interactions and fields of study such as game theory, social studies, operations research, engineering, and artificial intelligence. A constraint refers to the relationship between the state of objects, such as the constraint that the three angles of a triangle must sum to 180 degrees. Note that this constraint has not precisely stated each angle's value and still allows some flexibility. Said another way, the triangle constraint restricts the values that the three variables (each angle) can take, thus providing information that will be useful in finding values for the three angles.

Another example of a constrained problem comes from the recently-aired hit TV series *Buddies*, where a group of five (mostly mutual) friends would like to sit at a table with three chairs in specific arrangements at different times, but have requirements as to who they will and will not sit with.

Another example comes from scheduling: at the university level, there is a large number of classes that must be scheduled in various classrooms such that no professor or classroom is double booked. Further, there are some constraints on which classes can be scheduled for the same time, as some students will need to be registered for both.

Computers can be employed to solve these types of problems, but in general these tasks are computationally intractible and cannot be solved efficiently in all cases with a single algorithm [1]. However, by formalizing these types of problems in a constraint processing framework, we can identify classes of problems that can be solved using efficient algorithms.

### 1.1. Modelling

A **constraint satisfaction problem** (CSP) is formalized by a *constraint network*, which is the triple $\langle V,D,C\rangle$, where
- $V = \{V_i\}_{i=1}^n$ is the set of $n$ variables
- $D = \{D_i\}_{i=1}^n$ is the set of variable domains, where the domain of variable $V_k$ is $D_k$
- $C = \{C_i\}_{i=1}^m$ is the set of constraints on the values that each $V_i$ can take on. Specifically,
    - Each constraint $C_i = \langle S_i,R_i\rangle$ specifies allowed variable assignments.
    - $S_i \subset V$ contains the variables involved in the constraint, called the *scope* of the constraint.
    - $R_i$ is the constraint's *relation* and represents the simultaneous legal value assignments of variables in the associated scope.
        - For example, if scope of the first constraint is $S_1 = \{V_3, V_8\}$, then the relation $R_1$ is a subset of the Cartesian product of those variables' domains: $R_1 \subset D_3 \times D_8$, and an element of the relation $R_1$ could be written as a 2-tuple $(x,y)\in R_1$.

Note that for a CSP model, *any* consistent assignment of the variables (i.e., where all constraints are satisfied) constitutes a valid solution, it may not be the "best" solution. Notions of optimality can be captured by introducing an objective function which is used to find a valid solution with the lowest cost. This is referred to as a **constraint *optimization* problem** (COP). We will refer generally to CSPs with the understanding that a CSP can easily become a COP.

### 1.1.1. Modelling as a Graph
<div class="alert alert-danger">
<b>TODO:</b> Discuss the graphical view of constraints. Let's also assume that we are only considering binary CSPs and note that any n-connected CSP can be reformulated into an equivalent binary CSP.
</div>

### 1.2. Solving

The goal of formalizing a CSP as a constraint network model is to efficiently solve it using computational algorithms and tools. **Constraint programming** (CP) is a powerful tool to solve combinatorial constraint problems and is the study of computational systems based on constraints. Once the problem has been modeled as a formal CSP, a variety of computable algorithms could be used to find a solution that satisfies all constraints. We will restrict ourselves to using CP to solve CSPs with **discrete, finite domains**.

In general, there are two methods used to solve a CSP: search or inference. In previous 16.410/413 problems, **state-space search** was used to find the best path through some sort of graph or tree structure. Likewise, state-space search could be used to find a valid "path" through the CSP that satisfies each of the local constraints and is therefore a valid global solution. However, this approach would quickly become intractable as the number of variables and the size of each of their domains increase.

In light of this, the second solution method becomes more attractive. **Constraint propagation**, a specific type of inference, is used to reduce the number of legal values from a variable's domain by pruning values that would violate the constraints of the given variable. By making a variable locally consistent with its constraints, the domain of adjacent variables may potentially be further reduced as a result of missing values in the pairwise constraint of the two variables. In this way, by making the first variable consistent with its constraints, the constraints of neighboring variables can be re-evaluated, causing a further reduction of domains through the propagation of constraints. These ideas will later be formalized as $k$-consistency.

Constraint propagation may be combined with search, using the pros of both methods simultaneously. Alternatively, constraint propagation may be performed as a pre-processing pruning step so that search has a smaller state space to search over. Sometimes, constraint propagation is all that is required and a solution can be found without a search step at all.

After giving examples of modelling CSPs, this notebook will explore a variety of solution methods based on constraint propagation and search.

---

## 2. Modeling

Given a constrained problem, it is desirable to identify an appropriate constraint network model $\langle V,D,C\rangle$ that can be used to find its solution. Modelling for CSPs is an important step that can dramatically affect the difficulty in enumerating the associated constraints or efficiency of finding a solution.

### 2.1. Examples

In this section, we consider two puzzle problems and model them as CSPs.

<div class="alert alert-danger">
<b>TODO:</b> Make these models more formal?
</div>

### 2.1.1. Sudoku

Sudoku is a well-known puzzle that can be easily solved using the CSP framework. Given a $9\times 9$ grid partially filled with numbers from $1$ to $9$, the goal is to fill the grid entirely such that:

- each $3\times 3$ sub-grid contains all numbers from $1$ to $9$
- each row contains all numbers from $1$ to $9$
- each column contains all numbers from $1$ to $9$

<table><tr>
<td> <img src="images/start_sudoku.png" /> </td>
<td> <img src="images/end_sudoku.png" /> </td>
</tr></table>

### 2.1.2. Map Coloring

<div class="alert alert-danger">
TODO: Add map coloring figure
</div>

### 2.1.3. N-Queens

The N-Queens problem is a well-know puzzle among computer scientists and will be used as an example throughout this notebook. The problem statement is as follows: given any integer $N$, the goal is to place $N$ queens on an $N\times N$ chessboard satisfying the constraint that no two queens threaten each other. A queen can threaten any other queen that is on the same row, column, or diagonal.

<img src="images/nqueens.png" align="center"/>

---

## 3. Constraint Propagation Methods

As previously mentioned, the domain size of a CSP can be dramatically reduced by removing values from variable domains that would violate the relevant constraints. This idea is called **local consistency**. Treating each variable as a node in a graph and each constraint as an arc connecting two nodes, then enforcing local consistency  amounts to removing all inconsistent values from each variable's domain. In the case of Sudoku, enforcing local consistency throughout the puzzle yields the unique solution without the need for search.

### 3.2. Arc Consistency

The most well-known notion of local consistency is **arc consistency**, where the arc $\langle V_i, V_j \rangle$ between two variables $V_i$ and $V_j$ is said to be arc consistent if $\langle V_i, V_j \rangle$ and $\langle V_j, V_i \rangle$ are *directed* arc consistent.

The arc $\langle V_i, V_j \rangle$ is **directed arc consistent** (from $V_i$ to $V_j$) if $\forall a_i \in D_i \; 
\exists a_j \in D_j$ s.t. $\langle a_i, a_j \rangle \in C_{ij}$.

By making a CSP arc consistent, we are guaranteed that solutions to the CSP will be found in the reduced domain of the arc consistent CSP. However, we are not guaranteed that any assignment of variables from the reduced domain will offer a valid CSP solution. In other words, arc consistency is sound (all solutions are arc-consistent solutions) but incomplete (not all arc-consistent solutions are valid solutions).


#### 3.2.1. Algorithm: REVISE

<div class="alert alert-warning"><pre>
procedure REVISE(Vi,Vj)
  DELETE &lt;- false;
  for each X in Di do
    if there is no such Y in Dj such that (X,Y) is consistent,
    then
       delete X from Di;
       DELETE &lt;- true;
    endif;
  endfor;
  return DELETE;
end
</pre></div>

#### 3.2.2. Algorithm: AC-1

<div class="alert alert-warning"><pre>
procedure AC-1
  repeat
    for each cij in C do
      REVISE(xi, xj)
      REVISE(xj, xi)
    endfor
  until no domain is changed
end
</pre></div>

- Discuss complexity

#### 3.2.3. Algorithm: AC-3

<div class="alert alert-warning"><pre>
procedure AC-3
  for each cij in C do
    Q &lt;- Q union {&lt;xi,xj&gt;, &lt;xj,xi&gt;};
  endfor
  while Q not empty
    select and delete any arc (xi,xj) from Q;
    if REVISE(Vk,Vm) then
      Q &lt;- Q union {(xk,xi) | k neq i, k neq j }
    endif
  endwhile
end
</pre></div>

- Discuss complexity

<div class="alert alert-danger">
    <b>TODO:</b> Should we discuss other types of consistency?
</div>

### 3.3. Example

<div class="alert alert-danger">
    <b>TODO:</b> Show example of arc consistency with N-Queens
</div>

<!--
### 3.5. Example: integer domains with algebra

- Is this where **global constraints** can be discussed? It looks like that's how Russell and Norvig do it.
-->

---

## 4. Search Methods

Here we investigate using search methods for solving CSPs. As mentioned previously, a simple state-space search would be intractable as the number of variables and their domains increase. However, we will first examine state-space search in more detail and then move to a more clever search algorithm called backtrack search (BT) that checks consistency along the way.

We will use the map coloring example shown below to explain how to perform search.

<img src="images/search_mapcoloring.png" width="40%" />

### 4.1. Generic Search for CSPs

As we have studied before, a generic search problem can be specified by the following four elements: (1) state space, (2) initial states, (3) operator, and (4) goal test. In a CSP, consider the following definitions of these elements:
- state space
    - partial assignment to variables at the current iteration of the search
- initial state
    - no assignment
- operator
    - add a new assignment to any unassigned variable, e.g., $V_i = a$, where $a\in D_k$.
    - child extends parent assignments with new
- goal test
    - all variables are assigned
    - all constraints are satisfied
    
With this definition of a generic state-space search of CSP solutions, note that the branching factor is the sum of the domain size for all variables, $O(nk)$. The performance is exponential in the branching factor, $O([nk]^n)$.

With a better understanding of how expensive it can become to solve interesting problems with a simple state-space search, we are motivated to find a better searching algorithm.

### 4.2. Backtrack Search

Backtrack (BT) search

#### 4.2.1. Algorithm

<div class="alert alert-warning"><pre>
procedure BT(&lt;V,D,C&gt;)
  ; input: A constraint network, R = &lt;V,D,C&gt;
  ; output: A solution, or notification that the network is inconsistent

end
</pre></div>

#### 4.2.2. Example

<div class="alert alert-danger">
    <b>TODO:</b> Show example of BT with N-Queens
</div>

### 4.3. Branch and Bound

Suppose we would like to find the *best* solution (in some sense) to the CSP. This amounts to solving the associated constraint optimization problem (COP), where our constraint network is now a 4-tuple, $\langle X, D_X, C, f \rangle$, where $X\in D_X$, $C: D_X \to \{\operatorname{True},\operatorname{False}\}$ and $f: D_x\to\mathbb{R}$ is a cost function. We would like to find the variable assignments $X$ that solve
\begin{array}{ll@{}ll}
\text{minimize}  & f(X) &\\
\text{subject to}& C(X) &
\end{array}

By adding a cost function $f(X)$, we turn a CSP into a COP and we can use the branch and bound algorithm to find the solution with the lowest cost.

#### 4.3.2. Example

<div class="alert alert-danger">
    <b>TODO:</b> Can we add costs to the N-Queens problem to show a COP example using branch and bound?
</div>

---

## 5. Hybrid Methods

### 5.1. BT Search with Forward Checking (BT-FC)

### 5.2. BT-FC with Dynamic Variable Ordering

### 5.3. Adaptive Consistency: Bucket Elimination

---

# Symmetries

<img src="images/escher_2.jpg" align="center"/>
<div style="text-align: right"> M. C. Escher </div>


## Introduction

A CSP often exhibit some symmetries, which are mappings that preserve satisfiability of the CSP. They are particularly harmful when we are looking for all possible valid assignment of a CSP since symmetries have many solutions that are just a symmetric solution of another one.

> **Definition (Symmetry).** For any CSP instance $P = \langle X, D, C \rangle$, a solution symmetry of $P$ is a permutation of the set $X\times D$ that preserves the set of solutions to $P$.

In other words, a solution symmetry is a bijective mapping defined on the set of possible variable-value pairs of a CSP, that maps solutions to solutions.

## Why is symmetry important? 

A principal reason for identifying symmetry in CSPs is to reduce search effort by not exploring assignments that are symmetrically equivalent to assignments considered elsewhere in the search. Clearly, if the solution symmetry group is larger than the constraint symmetry group, there will potentially be a greater search reduction from using the solution symmetries, if they can somehow be identified in advance.


## Case Study: symmetries in N-Queens problem

The standard formulation of the N-Queens problem as a CSP has $N$ variables corresponding to the rows of the chessboard, say $Q_1,Q_2,\ldots,Q_N$. The domain of values corresponds to the columns of the chessboard, say $D = \{1, 2,\ldots, N\}$. The constraints can be expressed as follows:
- the values of $Q_1, Q_2, \ldots, Q_N$ are all different; 
- $\forall i,j,1\leq i< j \leq N,|Q_i−Q_j| \neq |i−j|$.

A chessboard has eight geometric symmetries: 
- horizontal reflection $r_h$
- vertical reflection $r_v$
- reflectins along the two diagonal axes ($r_{d_1}$ and $r_{d_2}$)
- rotations through $90$&deg;, $180$&deg; and $270$&deg; ($r_{90}$, $r_{180}$, $r_{270}$)
- identity (no-reflections) $r_I$

We can group all the symmetries in $G = \{r_I, r_h,r_v, r_{d_1}, r_{d_2},r_{90}, r_{180}, r_{270} \}$
For the seek of simplicity let's denote $A^g$, with $g\in G$ as the result of the application of symmetry $g$ to $A$. For example, in the 4-Queens problem $(Q_1 = 2)^{r_{90}} = (Q_2=4)$.

An example can be given by the 4-Queens problem, all the solutions are the following

<img src="images/two-queens_symmetries.png" align="center"/>

It's easy to note that the second solution is the first solution when we flip the chessboard horizontally.



## Avoid symmetries

### Adding Constraints Before Search

Symmetry in CSPs is usually identified, in practice, by applying human insight: the programmer sees that some transformation would transform a hypothetical solution into another hypothetical solution. Then the programmer can try to formalize some constraint that preserve solutions but remove some of the symmetries.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of N-Queens problem with broken symmetries through additional constraints
</div>

This approach is fine if done correctly, but can obviously lose solutions if done incorrectly.


### Dynamic Symmetry Breaking Methods

Dynamic symmetry breaking methods are those that operate to break symmetry during the search process, avoiding some brach of the search tree that are symmetrically equivalent to others. Let's explore some of them.

#### Symmetry Breaking During Search (SBDS)

The basic idea of SBDS (this is the name of the algorithm, not a class for them) is to add constraints to a problem so that, after backtracking from a search decision, the SBDS constraints ensure that no symmetric equivalent of that decision is ever allowed. This is a dynamic technique, since we cannot add the constraints until we know what search decision is being made.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example search tree for SBDS
</div>

A big drawback of this agorithm is that it always demand from the constraint programmer to explicitly state all symmetries $g\in G$a. If a problem has a large number of symmetries there may be too many for the user to identify and implement by hand.

### The Lex-Leader Method

Puget proved that whenever a CSP has symmetry [2], it is possible to find a _'reduced form'_, with the symmetries eliminated, by adding constraints to the original problem. Puget found such a reduction for three simple constraint problems, and showed that this reduced CSP could be solved more efficiently than in its original form. From this work a method called **lex-leader** was developed, the background idea is rather simple, for each equivalence class of solutions under our symmetry group, we will predefine one to be the canonical solution. We will achieve this by choosing a static variable ordering. 

For example let's consider a problem where we have three variables $x_1$ $x_2$ and $x_3$ subject to `alldifferent` constraint and domain $\{1,2,3\}$. This problem has $3!$ solutions, the easiest way to impose a canonical solution is to impose that $x_1 \leq x_2 \leq x_3$, that is the _lexicographic_ ordering. Adding this additional constraint we will get exactly one solution. All the other can be recovered applying all the symmetries.

In general the lex-leader method impose a variable ordering for Each permutation in the symmetry group, formally

$$ \forall g \in G, \, A\preceq_{\text{lex }} A^g $$

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of model with/without lex-leader
</div>

An important practical issue with the lex-leader constraints is that they do not “respect” the variable and value ordering heuristics used in search. That is, it may well be that the leftmost solution in the search tree, which would otherwise be found first, is not canonical and so is disallowed, leading to increased search. This is in contrast to techniques such as SBDS, which do respect the heuristic.

### Double-Lex

A matrix of decision variables has row symmetry iff given a solution, any permutation of the rows is also a solution. Similarly, it has column symmetry iff given a solution, any permutation of the columns is also a solution.  Row and column symmetry occurs in many mod- els with matrices of decision variable. We have seen that that we can add $n!m!$  constraints to break all the symmetries. Adding so many constraints can be counter-productive. A common approach is order the rows of a matrix and **independently** order the columns. This produces only $n + m − 2$ symmetry breaking constraints. This method is called **double-lex** and has the disadvantage of breaking only some of the symmetries.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Example of the same model as before but with double-lex
</div>

## Intractability of Breaking Symmetry


It is worth to mention that lex-leader requires one constraint for each element of the group. 
In the case of a matrix with $m$ rows and $n$ columns, this is $m!n!$, which is impractical in general. Therefore there are many cases where lex-leader is applicable but impractical.

> **Theorem (Walsh 2011)** Given any _simple_ ordering, there exists a symmetry group such that deciding if an as- signment is smallest in its symmetry class according to this ordering is NP-hard.

Since breaking symmetry appears intractable in general, a major research direction is to identify special cases which occur in practice where the symmetry group is more tractable to break.

> **Theorem (Katsirelos, Narodytska, and Walsh 2010)** Propagating the double-lex constraint is NP-hard.

## Partial Symmetry Breaking

If we are trying to solve a problem that is exponentially symmetric we may not be able to fully utilize a given symmetry breaking technique.

By only describing a subset of symmetries we are performing partial symmetry breaking (PSB) i.e. performing some redundant search because the symmetry breaking technique is too costly
or even impossible to perform. 

The problem with this approch is that is it hard to select wih simmetry constraint to remove, for this reason there are a class of algorithms that use heuristics to 

## Symmetry Breaking by Nonstationary Optimisation

Suppose that we are solving a problem using depth-first search (DFS) with static variable ordering, and constraint processing.
If we can apply a group element $g ∈ G$ to the current partial assignment $A$ such that $A^g\prec_{\text{lex }} A$, then under the above assumptions $A^g$ dominates (symmetrically equivalent to) $A$ and we can backtrack from $A$.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Graphical example
</div>

### Detection as nonstationary optimisation

We can model the detection problem as an optimisation problem with $G$ as the search space, so that each $g ∈ G$ is a search state. The objective function of $g$ to be minimised is the lex-ranking of $A^g$. On finding an element $g$ with sufficiently small objective value we have solved the detection problem.

### Detection by local search

To make SBNO more concrete we can use local search for detection. 
In this contex $G$ is the search space the lex-ranking of $A^g$ is the objective function. Local search requires a neighbourhood structure defining the possible local moves from each search state. 

To impose a neighbourhood structure on $G$ we choose some subset $H \subset G$: from any search state $g$ the possible local moves are the elements of $H$ leading to neighbouring states $g \odot H$. Thus all $G$ elements are local search states, and some of them ($H$) are also local moves. 


To apply hill climbing, from each state g we try to find a local move $h$ such that the objective function is reduced ($A^{g\odot h} \prec_{\text{lex}} A^g$). If a series of moves $h_1, h_2, \ldots$ reduces the lex-ranking sufficiently then we will find $A^{g\odot h_1\odot h_2 \odot\ldots} \prec_{\text{lex}} A$ and can backtrack from $A$.

<div class="alert alert-block alert-danger">
<b>TODO:</b> Algorithm pseudocode
</div>

## Reduce the set of Symmetry breaking constraints

When we are dealing with symmetries we always have a tradeoff between the size of the search tree and the propagation effort. Using lex-leader adds too many contraints, double-lex is better but might leave a lot of symmetries.

We can think of an extension of double-lex that breaks more symmetries, examples are:

- $\text{lex}^+$ also called _swap-next_, considers a pair of adjacent rows and a pair of adjacent columns
    - $mn -1 $ constraints
- $\text{lex}^*$ also called _swap_any_, considers any pair of rows and any pair of columns
    - $O(m^2n^2)$ constraints

### Lex constraint decomposition

We can decompose the lex constraint of the form $x_1\ldots x_2 \preceq_{\text{lex }} y_1\ldots y_2$ to a conjunction of clauses like
$$ \left(x_1 = y_1 \right),\ldots,\left(x_k = y_k\right)\rightarrow x_{k+1}\leq y_{k+1} $$
We call clauses of this form lex implications.

According to [4] roughly 65% of the lex implications of $\text{lex}^*$ are redundant. So we can try to develop an algorithm that tries to remove all redundant constraints. Given this observation, we can pose pose the direct question: how many lex implications are required to express a complete symmetry break.

### Reduce implications

Basically, the algorithm iterates over the set of lex implications, and checks for each of them if they are redundant. This is done by removing them from the formula, and checking if there is a solution which would be forbidden by this clause, as shown in Algorithm 2. If this is not the case, the clause is redundant and can be removed. This is accomplished by running a SAT solver on the implication.

The reduction splits in two phases. 

1. The first phase is shown in Algorithm 1. We check if a clause $c$ is redundant. If this is the case, we compute a subset $\psi\subseteq\varphi^\prime$ of clauses which makes $c$ redundant, and increase the ranking of all clauses within this set. The rationale is that removing these clauses is more likely make other clauses no longer redundant, and so increase the size of the final symmetry break.
2. In the second stage, we sort the clauses by ranking, so clauses which were frequently the cause of redundancy appear as late as possible. We then reduce the set of lex implications using Algorithm 2.

<div class="alert alert-warning">
<img src="images/rm_symmetries.png" align="center"/>
</div>

<div class="alert alert-danger">
<b>TODO:</b> fix notation, \phi is the set of constraints
</div>

### Example

The number of solutions for Boolean matrix models with various symmetry breaks.

| n | None | $\text{lex}^2$ |  $\text{lex}^+$ | $\text{lex}^*$ | Complete |
|:--|-----:|---------------:|----------------:|---------------:|---------:|
| 3 |$2^9$| 45 | 37 | 36 | 36 | 
| 4 |$2^{16}$| 650 | 366 | 330 | 317 | 
| 5 |$2^{25}$| 24520 | 8659 | 6779 | 5624 |
| 6 |$2^{36}$| 2625117 | 602813 | 391532 | 251610 |
| 7 |$2^{49}$| 836488618 | 139268908 | 73720859 | 33642660 |

Number of lex implications for  $\text{lex}^*$ for matrix models before and after reduction.

| n | Before | After |
|:--|-----:|--------:|
| 3 | 64
| 4 | 294
| 5 | 968
| 6 | 2560  | 
| 7 | 5808  |
| 8 | 11774 |
| 9 | 21904 |
| 10| 38088 |


# References

- [1] Rina Dechter. 2003. Constraint Processing. Morgan Kaufmann Publishers Inc., San Francisco, CA, USA.
- [2] J.-F. Puget. On the satisfiability of symmetrical constraint satisfaction problems. In Methodologies for Intelligent Systems (Proceedings of ISMIS’93), LNAI 689, pages 350–361. Springer, 1993.
- [3] https://ktiml.mff.cuni.cz/~bartak/constraints/index.html
- [4] Codish M., Ehlers T., Gange G., Itzhakov A., Stuckey P.J. (2018) Breaking Symmetries with Lex Implications. In: Gallagher J., Sulzmann M. (eds) Functional and Logic Programming. FLOPS 2018. Lecture Notes in Computer Science, vol 10818. Springer, Cham